### イベント基礎データ取得

In [148]:
import requests
from typing import Any, Dict, List, Optional, Tuple
from datetime import datetime, timedelta, timezone, time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

BASE_URL = "https://api.sekai.best"
REGION = "jp"
EVENTS_JSON_URL = (
    "https://raw.githubusercontent.com/Sekai-World/sekai-master-db-diff/"
    "main/events.json"
)
JST = timezone(timedelta(hours=9))

#### イベントデータ一覧取得

In [64]:
def fetch_event_list():
    resp = requests.get(EVENTS_JSON_URL)
    resp.raise_for_status()
    data = resp.json()
    if not isinstance(data, list):
        raise ValueError("events.json の中身がリストではありません。")
    return data

In [65]:
def list_event_names():
    events = fetch_event_list()
    event_names = [e["name"] for e in events if isinstance(e, dict) and "name" in e]
    print("\n".join(event_names))

In [97]:
def get_event_info_by_name(event_name):
    events = fetch_event_list()
    for evt in events:
        if not isinstance(evt, dict):
            continue
        if evt.get("name") == event_name:
            return evt
    raise ValueError(f"イベント名【{event_name}】が見つかりませんでした。")

In [160]:
def filter_event_info(evt):
    return evt.get("id"), datetime.fromtimestamp(evt.get("startAt") / 1000, tz=JST), datetime.fromtimestamp(evt.get("aggregateAt") / 1000, tz=JST)

#### イベント名から ID と開催期間を抽出

In [161]:
filter_event_info(get_event_info_by_name("いつか花咲くステージへ"))

(130,
 datetime.datetime(2024, 5, 9, 20, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400))),
 datetime.datetime(2024, 5, 21, 19, 59, 59, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400))))

---
### まずは通常イベントのランキング取得

In [69]:
def get_event_time(event_id):
    url = f"{BASE_URL}/event/{event_id}/rankings/time"
    params = {
        "region": REGION
    }
    try:
        resp = requests.get(url, params=params, timeout=10)
        print("Status:", resp.status_code)
        print("URL:", resp.url)
        resp.raise_for_status()
        payload = resp.json()
        data = payload.get("data", [])
        return data if isinstance(data, list) else []
    except Exception as e:
        print("Error:", e)
        return []

In [72]:
event_id, _, _ = filter_event_info(get_event_info_by_name("いつか花咲くステージへ"))
times = get_event_time(event_id)

Status: 200
URL: https://api.sekai.best/event/130/rankings/time?region=jp


In [73]:
def get_event_rankings(event_id, ts):
    url = f"{BASE_URL}/event/{event_id}/rankings"
    params = {"timestamp": ts, "region": REGION}
    try:
        resp = requests.get(url, params=params, timeout=100)
        print("Status:", resp.status_code)
        print("URL:", resp.url)
        resp.raise_for_status()
        payload = resp.json()
        data = payload.get("data")
        return data.get("eventRankings")
    except Exception as e:
        print("Error:", e)
        return []

In [74]:
get_event_rankings(event_id, times[0])

Status: 200
URL: https://api.sekai.best/event/130/rankings?timestamp=2024-05-10T08%3A39%3A00.549Z&region=jp


[{'id': 30237160,
  'eventId': 130,
  'timestamp': '2024-05-10T08:39:00.549Z',
  'rank': 1,
  'score': 54151652,
  'userId': '8343521107030030',
  'userName': '愛莉ちゃんカワイイな',
  'userCard': {'level': 60,
   'cardId': 424,
   'masterRank': 5,
   'defaultImage': 'special_training',
   'specialTrainingStatus': 'done'},
  'userProfile': {'word': 'みずえなしずあい',
   'userId': '8343521107030030',
   'twitterId': 'mamemamemame_p',
   'profileImageType': 'leader'},
  'userCheerfulCarnival': {},
  'userProfileHonors': [{'seq': 1,
    'honorId': 1072002,
    'honorLevel': 10,
    'bondsHonorWordId': 1072002,
    'profileHonorType': 'bonds',
    'bondsHonorViewType': 'normal'},
   {'seq': 3,
    'honorId': 3411,
    'honorLevel': 1,
    'bondsHonorWordId': 0,
    'profileHonorType': 'normal',
    'bondsHonorViewType': 'none'}]},
 {'id': 30237161,
  'eventId': 130,
  'timestamp': '2024-05-10T08:39:00.549Z',
  'rank': 2,
  'score': 52759431,
  'userId': '8095312367759366',
  'userName': 'むっ♡',
  'userCard'

---
### WL に対応させるため event_id とチャプター番号から character_id を抽出

In [46]:
WORLD_BLOOM_JSON_URL = (
    "https://raw.githubusercontent.com/Sekai-World/sekai-master-db-diff/"
    "main/worldBlooms.json"
)
def fetch_world_bloom():
    resp = requests.get(WORLD_BLOOM_JSON_URL)
    resp.raise_for_status()
    data = resp.json()
    if not isinstance(data, list):
        raise ValueError("events.json の中身がリストではありません。")
    return data

In [50]:
def get_game_character_id(event_id, chapter_no):
    items = fetch_world_bloom()
    return next(
        (o.get("gameCharacterId")
         for o in items
         if o.get("eventId") == event_id and o.get("chapterNo") == chapter_no),
        None
    )

In [77]:
chara_id = get_game_character_id(130, 1)

#### WL chapter 単位のランキング取得

In [78]:
def get_chapter_time(event_id, chara_id):
    url = f"{BASE_URL}/event/{event_id}/chapter_rankings/time"
    params = {"charaId": chara_id, "region": REGION}
    try:
        resp = requests.get(url, params=params, timeout=10)
        print("Status:", resp.status_code)
        print("URL:", resp.url)
        resp.raise_for_status()
        payload = resp.json()
        data = payload.get("data", [])
        return data if isinstance(data, list) else []
    except Exception as e:
        print("Error:", e)
        return []

In [79]:
event_id, _, _ = filter_event_info(get_event_info_by_name("いつか花咲くステージへ"))
times = get_chapter_time(event_id, chara_id)

Status: 200
URL: https://api.sekai.best/event/130/chapter_rankings/time?charaId=7&region=jp


In [80]:
def get_chapter_rankings(event_id, chara_id, ts):
    url = f"{BASE_URL}/event/{event_id}/chapter_rankings"
    params = {"charaId": chara_id, "timestamp": ts, "region": REGION}
    try:
        resp = requests.get(url, params=params, timeout=100)
        print("Status:", resp.status_code)
        print("URL:", resp.url)
        resp.raise_for_status()
        payload = resp.json()
        data = payload.get("data")
        return data.get("eventRankings")
    except Exception as e:
        print("Error:", e)
        return []

In [81]:
rankings = get_chapter_rankings(event_id, chara_id, times[0])

Status: 200
URL: https://api.sekai.best/event/130/chapter_rankings?charaId=7&timestamp=2024-05-09T11%3A03%3A00.274Z&region=jp


In [82]:
rankings

[{'id': 132120,
  'eventId': 130,
  'timestamp': '2024-05-09T11:03:00.274Z',
  'rank': 1,
  'score': 88095,
  'userId': '12150586078896140',
  'userName': '☀ぱっぱらぱむらんど',
  'userCard': {'level': 60,
   'cardId': 424,
   'masterRank': 5,
   'defaultImage': 'original',
   'specialTrainingStatus': 'done'},
  'userProfile': {'word': 'ぱっぱらぱむらんど！！！！',
   'userId': '12150586078896140',
   'twitterId': 'pamuun_prsk',
   'profileImageType': 'leader'},
  'userCheerfulCarnival': {},
  'userProfileHonors': [{'seq': 1,
    'honorId': 27,
    'honorLevel': 3,
    'bondsHonorWordId': 0,
    'profileHonorType': 'normal',
    'bondsHonorViewType': 'none'},
   {'seq': 2,
    'honorId': 19,
    'honorLevel': 2,
    'bondsHonorWordId': 0,
    'profileHonorType': 'normal',
    'bondsHonorViewType': 'none'},
   {'seq': 3,
    'honorId': 23,
    'honorLevel': 1,
    'bondsHonorWordId': 0,
    'profileHonorType': 'normal',
    'bondsHonorViewType': 'none'}]},
 {'id': 132121,
  'eventId': 130,
  'timestamp': '20

---
### Google spread sheet への書き込みを考える

In [122]:
import gspread
from google.oauth2.service_account import Credentials
from gspread.utils import rowcol_to_a1
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]
creds = Credentials.from_service_account_file("./keys/rock-perception-419201-eb5dbe72985b.json", scopes=SCOPES)
gc = gspread.authorize(creds)
SPREADSHEET_ID = "1ySZ0ja5_K1ak6mFCL3n-N_-mS_KDMsmFv8TeEm2TQCo"
SHEET_NAME = "Config"

In [92]:
sh = gc.open_by_key(SPREADSHEET_ID)
ws = sh.worksheet(SHEET_NAME)

In [93]:
values = ws.get("A1:C10")
print("READ:", values)

READ: [['EventName'], ['ChapterNo'], ['Runners']]


In [112]:
def read_config_values(spreadsheet_id: str,
                       sheet_name: str = "Config") -> dict:
    ws = gc.open_by_key(spreadsheet_id).worksheet(sheet_name)
    event_name = (ws.acell("B1").value or "").strip()
    raw_chapter = (ws.acell("B2").value or "").strip()
    chapter_no = None
    if raw_chapter:
        try:
            chapter_no = int(raw_chapter)
        except ValueError:
            try:
                chapter_no = int(float(raw_chapter))
            except Exception:
                chapter_no = None
                
    row3 = ws.row_values(3)
    runners = [v.strip() for v in row3[1:] if isinstance(v, str) and v.strip()]
    return {
        "EventName": event_name,
        "ChapterNo": chapter_no,
        "Runners": runners,
    }

In [113]:
config = read_config_values(SPREADSHEET_ID)

In [114]:
config.get("EventName")

'All ways Jump! with you'

In [159]:
get_event_info_by_name(config.get("EventName"))

{'id': 176,
 'eventType': 'world_bloom',
 'name': 'All ways Jump! with you',
 'assetbundleName': 'event_wl_2nd_idol_2025',
 'bgmAssetbundleName': 'event/event_wl_2nd_idol_2025/bgm/event_wl_2nd_idol_2025_top',
 'eventPointAssetbundleName': 'icon_point',
 'eventOnlyComponentDisplayStartAt': 1754622000000,
 'startAt': 1754650800000,
 'aggregateAt': 1755687599000,
 'rankingAnnounceAt': 1755688199000,
 'distributionStartAt': 1755741599000,
 'eventOnlyComponentDisplayEndAt': 1755831599000,
 'closedAt': 1755842399000,
 'distributionEndAt': 1756997999000,
 'virtualLiveId': 393,
 'unit': 'idol',
 'eventRankingRewardRanges': [{'id': 5309,
   'eventId': 176,
   'fromRank': 1,
   'toRank': 1,
   'isToRankBorder': False,
   'eventRankingRewards': [{'id': 5309,
     'eventRankingRewardRangeId': 5309,
     'resourceBoxId': 5309}]},
  {'id': 5310,
   'eventId': 176,
   'fromRank': 2,
   'toRank': 2,
   'isToRankBorder': False,
   'eventRankingRewards': [{'id': 5310,
     'eventRankingRewardRangeId': 5

In [162]:
event_id, start, end = filter_event_info(get_event_info_by_name(config.get("EventName")))

In [163]:
event_id

176

In [164]:
start

datetime.datetime(2025, 8, 8, 20, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400)))

In [165]:
end

datetime.datetime(2025, 8, 20, 19, 59, 59, tzinfo=datetime.timezone(datetime.timedelta(seconds=32400)))

In [167]:
def format_shift_table(
    spreadsheet_id: str,
    start: datetime,
    end: datetime,
    sheet_title: str = "Shift",
    service_account_json: str = "service_account.json",
) -> str:
    if start > end:
        raise ValueError("start must be <= end")
    sh = gc.open_by_key(spreadsheet_id)
    start_h = start.replace(minute=0, second=0, microsecond=0)
    end_h   = end.replace(minute=0, second=0, microsecond=0)

    days = []
    d = start.date()
    end_d = end.date()
    while d <= end_d:
        days.append(d)
        d += timedelta(days=1)

    total_rows = 25
    total_cols = (1 + (len(days) - 1) * 5 + 4) if days else 5
    try:
        ws = sh.add_worksheet(title=sheet_title, rows=total_rows, cols=total_cols)
    except gspread.exceptions.APIError:
        ws = sh.add_worksheet(
            title=f"{sheet_title}_{int(datetime.now().timestamp())}",
            rows=total_rows,
            cols=total_cols,
        )

    for i, day in enumerate(days):
        col = 1 + i * 5  # A=1, F=6, K=11 ...
        if day == start_h.date():
            day_start_hour = start_h.hour
        else:
            day_start_hour = 0
        if day == end_h.date():
            day_end_hour = end_h.hour
        else:
            day_end_hour = 23

        hours_col = []
        for h in range(24):
            hours_col.append([f"{h:02d}:00"] if day_start_hour <= h <= day_end_hour else [""])

        values = [[day.strftime("%Y-%m-%d")]] + hours_col  # 25x1
        cell_range = f"{rowcol_to_a1(1, col)}:{rowcol_to_a1(25, col)}"
        full_range = f"{ws.title}!{cell_range}"
        sh.values_update(
            full_range,
            params={"valueInputOption": "USER_ENTERED"},
            body={"values": values},
        )

    try:
        ws.freeze(rows=1)
    except Exception:
        pass

    return ws.title

In [168]:
created_title = format_shift_table(SPREADSHEET_ID, start, end)